In [1]:
import numpy as np
from stable_baselines3 import PPO
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from environments import MultiMarkEnv

2024-01-19 14:25:53.754230: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-19 14:25:53.772519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 14:25:53.772536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 14:25:53.773033: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 14:25:53.776465: I tensorflow/core/platform/cpu_feature_guar

In [2]:
outer_radius = 2 * 250 + 2.5 * 0.1 * 250

dt = 5
num_marks = 2
max_seconds = 500 * num_marks / dt

# Initialize the figure and scatter plot
fig = go.FigureWidget()
scatter = fig.add_scatter(mode='markers+lines', name='trajectory')
marks_scatter = fig.add_scatter(mode='markers+text', name='marks')
fig.update_xaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.update_yaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.layout.width=800
fig.layout.height=800

heading_fig = go.FigureWidget()
heading_scatter = heading_fig.add_scatter(mode='markers+lines', name='heading')
rudder_angle_scatter = heading_fig.add_scatter(mode='markers+lines', name='rudder angle')
heading_fig.update_xaxes(range=[0, max_seconds],dtick=25)
heading_fig.update_yaxes(range=[-210, 210],dtick=30)
heading_fig.layout.width=800
heading_fig.layout.height=600
heading_fig.layout.title = "Heading over time"

vmg_fig = go.FigureWidget()
vmg_scatter = vmg_fig.add_scatter(mode='markers+lines')
vmg_fig.update_xaxes(range=[0, max_seconds],dtick=25)
vmg_fig.update_yaxes(range=[-4, 4],dtick=2)
vmg_fig.layout.width=800
vmg_fig.layout.height=400
vmg_fig.layout.title = "VMG over time"

# Initialize output for the text
out = widgets.Output()

# Display text and figure
display(out)
display(fig)
display(heading_fig)
display(vmg_fig)

episode = 0

colormap = ['blue', 'red', 'green', 'orange', 'black']

def plot(data, marks, bounds):
    global out, fig
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds

    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    rudder_angles = [point['meta']['rudder_angle'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum([point['meta']['reward'] for point in data]), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    rudder_angle_scatter = heading_fig.data[1]
    rudder_angle_scatter.x = list(range(len(rudder_angles)))
    rudder_angle_scatter.y = rudder_angles
    rudder_angle_scatter.marker.color = 'lightgreen'
    rudder_angle_scatter.line.color = 'lightgreen'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")

None

Output()

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'trajectory',
              'type': 'scatter',
              'uid': 'be77e928-1e52-43c7-b8b5-e71ae3d021b2'},
             {'mode': 'markers+text', 'name': 'marks', 'type': 'scatter', 'uid': 'ecb7e92f-fe80-4f58-953a-94f9c9a124ee'}],
    'layout': {'height': 800,
               'template': '...',
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [-562.5, 562.5]},
               'yaxis': {'dtick': 25, 'range': [-562.5, 562.5]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'heading',
              'type': 'scatter',
              'uid': '14e8c148-6d2c-4280-8c41-9bd778737168'},
             {'mode': 'markers+lines',
              'name': 'rudder angle',
              'type': 'scatter',
              'uid': 'cedaa24a-9748-43ae-9b6e-72f5ee9abd92'}],
    'layout': {'height': 600,
               'template': '...',
               'title': {'text': 'Heading over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 200.0]},
               'yaxis': {'dtick': 30, 'range': [-210, 210]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines', 'type': 'scatter', 'uid': '3465017f-2253-4a18-9cc0-95e87c7804c5'}],
    'layout': {'height': 400,
               'template': '...',
               'title': {'text': 'VMG over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 200.0]},
               'yaxis': {'dtick': 2, 'range': [-4, 4]}}
})

In [3]:
# Configuration

r = 250
actions = np.array([-90, -45, -15, 0, 15, 45, 90]) / 180.0

config = {
    'max_marks': 1,
    'max_seconds_per_leg': 500,
    'plot_fn': lambda x, y, z: None,
    'leg_radius': r,
    'actions': actions,
    'target_tolerance_multiplier': 2
}

outer_radius = config['max_marks'] * r + 0.2 * r
bounds = [-outer_radius, outer_radius, -outer_radius, outer_radius]
# Initialize Environment
env = MultiMarkEnv(config, dt=5, bounds=bounds)

# Initialize PPO model
model = PPO.load('./MultiMarkPPO_boat_model_fine.dat')

In [4]:
targets = [(0, 250), (250, 250), (250, 0), (0, 0), (-250, 0), (-250, -250), (0, -250), (0, 0)]

full_trajectory = []

start_x = 0
start_y = 0
start_heading = np.pi/4

for mark_idx, (target_x, target_y) in enumerate(targets):
    best_trajectory = None
    best_reward = None
    for i in range(20):
        obs, _ = env.reset()
        env.target_x[0] = target_x - start_x
        env.target_y[0] = target_y - start_y
        env.boat.heading = start_heading
        env.heading = start_heading
        done = False
        total_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, is_terminal, is_truncated, info = env.step(action)
            done = is_terminal or is_truncated
            total_reward += reward

        if best_reward is None or total_reward > best_reward:
            best_reward = total_reward
            best_trajectory = env.trajectory

    for item in best_trajectory:
        item['x'] += start_x
        item['y'] += start_y
        item['meta']['current_mark'] = mark_idx

    full_trajectory.extend(best_trajectory)
    start_x = env.x + start_x
    start_y = env.y + start_y
    start_heading = env.boat.heading

plot(full_trajectory, targets, bounds)

In [5]:
full_trajectory

[{'x': 0.0,
  'y': 0.0,
  'meta': {'current_mark': 0,
   'vmg': 0,
   'heading': -151.276855831782,
   'rudder_angle': -45.00000000000006,
   'reward': 0}},
 {'x': 9.508435049190206,
  'y': 11.269881958465355,
  'meta': {'current_mark': 0,
   'vmg': 2.347312394435457,
   'heading': 36.037120572846256,
   'rudder_angle': -45.00000000000006,
   'reward': 0.23473123944354568}},
 {'x': 16.47152987438489,
  'y': 23.31768351307344,
  'meta': {'current_mark': 0,
   'vmg': 2.2754625117249097,
   'heading': 26.008329786904387,
   'rudder_angle': -45.00000000000006,
   'reward': 0.22754625117249097}},
 {'x': 21.786363192470642,
  'y': 35.156127941662234,
  'meta': {'current_mark': 0,
   'vmg': 2.252618404295266,
   'heading': 22.956898251898245,
   'rudder_angle': -15.0,
   'reward': 0.22526184042952663}},
 {'x': 26.796406491437388,
  'y': 46.983780529700475,
  'meta': {'current_mark': 0,
   'vmg': 2.2140623439064178,
   'heading': 22.956898251898245,
   'rudder_angle': 0.0,
   'reward': 0.22140